# Coleta de Dados do Sentinel-2 usando Google Earth Engine

Este notebook demonstra como coletar dados do Sentinel-2 para áreas de interesse definidas em arquivos KML.

In [ ]:
import sys
sys.path.append('..')

import ee
from pathlib import Path
from src.data.gee_collector import SentinelCollector
from src.data.geometry_handler import GeometryHandler

# Inicializar Earth Engine
ee.Initialize()

## 1. Configurar o Coletor de Dados

Primeiro, vamos configurar o SentinelCollector com os diretórios necessários:

In [ ]:
# Configurar diretórios
data_dir = Path('../data')
kml_dir = data_dir / 'kmls'
output_dir = data_dir / 'tiffs'

# Criar diretórios se não existirem
kml_dir.mkdir(parents=True, exist_ok=True)
output_dir.mkdir(parents=True, exist_ok=True)

# Inicializar o coletor
collector = SentinelCollector(
    output_dir=str(output_dir),
    geometry_handler=GeometryHandler()
)

## 2. Coletar Dados para Treinamento e Validação

Vamos coletar dados usando KMLs separados para áreas de treinamento e validação:

In [ ]:
# Definir caminhos dos KMLs
train_kml = str(kml_dir / 'areas_treino.kml')
val_kml = str(kml_dir / 'areas_validacao.kml')

# Carregar regiões dos KMLs
train_regions = collector.load_regions(train_kml)
val_regions = collector.load_regions(val_kml)

# Coletar imagens do Sentinel-2
train_collection = collector.get_sentinel_collection(
    regions=train_regions,
    years_back=2,  # Últimos 2 anos de dados
    max_cloud_cover=20  # Máximo de 20% de cobertura de nuvens
)

val_collection = collector.get_sentinel_collection(
    regions=val_regions,
    years_back=2,
    max_cloud_cover=20
)

## 3. Exportar para GeoTIFF

Agora vamos exportar as imagens coletadas para arquivos GeoTIFF:

In [ ]:
# Configurar parâmetros de exportação
export_params = {
    'scale': 10,  # Resolução de 10 metros
    'patch_size': 64,  # Tamanho do patch em pixels
    'bands': ['B8', 'B4', 'B11']  # Bandas relevantes para vegetação
}

# Exportar dados de treino
collector.export_to_drive(
    collection=train_collection,
    regions=train_regions,
    output_prefix='treino',
    **export_params
)

# Exportar dados de validação
collector.export_to_drive(
    collection=val_collection,
    regions=val_regions,
    output_prefix='validacao',
    **export_params
)

## 4. Monitorar Exportação

As tarefas de exportação serão executadas no Google Earth Engine. Você pode monitorar o progresso no painel de tarefas do Earth Engine (https://code.earthengine.google.com/tasks).

Após a conclusão, os arquivos estarão disponíveis no seu Google Drive na pasta especificada.

## 5. Próximos Passos

Depois que os dados forem exportados:
1. Baixe os arquivos GeoTIFF do Google Drive
2. Mova-os para o diretório apropriado no projeto
3. Use o TiffLoader para carregar os dados no formato adequado para treinamento

Importante: Certifique-se de que você tem:
- Acesso configurado ao Google Earth Engine
- Espaço suficiente no Google Drive
- KMLs bem definidos para suas áreas de interesse